<a href="https://colab.research.google.com/github/CristianDavid010/ejercicioOftalmologia/blob/main/Oftalmologia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ANÁLISIS DE RIESGO DE GLAUCOMA CON RANDOM FOREST
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import (classification_report, confusion_matrix,
                            ConfusionMatrixDisplay, roc_auc_score)
import matplotlib.pyplot as plt

# 1. Datos de glaucoma (variables clínicas clave)
# Simulación de datos (reemplazar con dataset real)
data = pd.DataFrame({
    'Edad': [65, 72, 58, 41, 68, 55, 70],  # Factor de riesgo importante
    'PIO': [22, 28, 18, 16, 25, 19, 30],   # Presión intraocular (mmHg)
    'Paquimetria': [520, 480, 550, 570, 490, 530, 460],  # Grosor corneal (µm)
    'RNFL': [88, 72, 85, 92, 68, 80, 65],  # Grosor capa fibras nerviosas (µm)
    'Campo_Visual': [-2.5, -6.8, -1.2, -0.5, -8.5, -3.0, -10.2],  # Pérdida en dB
    'Glaucoma': [1, 1, 0, 0, 1, 0, 1]      # Diagnóstico (1: Sí, 0: No)
})

X = data.drop('Glaucoma', axis=1)
y = data['Glaucoma']

# 2. División de datos (stratified para mantener proporción clínica)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                  stratify=y,
                                                  random_state=42)

# 3. Búsqueda de hiperparámetros con enfoque clínico
param_grid = {
    'n_estimators': [50, 100, 150],       # Número de árboles
    'max_depth': [3, 5, None],            # Profundidad (None para ilimitado)
    'min_samples_split': [2, 5, 10],      # Mínimo para dividir nodo
    'class_weight': [{0: 1, 1: 2}]        # Mayor peso en casos positivos (glaucoma)
}

model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3,
                         scoring='recall',  # Enfasis en no falsos negativos
                         n_jobs=-1)
grid_search.fit(X_train, y_train)

# 4. Evaluación del modelo óptimo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilidades para curva ROC

print("🔍 HIPERPARÁMETROS ÓPTIMOS PARA GLAUCOMA:")
print(grid_search.best_params_)

print("\n📋 INFORME CLÍNICO (Test Set):")
print(classification_report(y_test, y_pred,
                           target_names=["Sin Glaucoma", "Glaucoma"]))

print(f"📊 AUC-ROC: {roc_auc_score(y_test, y_proba):.2f}")

# 5. Visualizaciones médicas
fig, ax = plt.subplots(1, 3, figsize=(18, 5))

# A. Matriz de confusión clínica
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=["Sin Glaucoma", "Glaucoma"])
disp.plot(ax=ax[0], cmap='Blues', colorbar=False)
ax[0].set_title("Validación Diagnóstica")

# B. Importancia de factores de riesgo (personalizado)
features_glaucoma = {
    'PIO': 'Presión Intraocular',
    'RNFL': 'Grosor RNFL',
    'Campo_Visual': 'Pérdida Campo Visual',
    'Edad': 'Edad',
    'Paquimetria': 'Paquimetría'
}

importances = pd.Series(best_model.feature_importances_, index=X.columns)
importances = importances.rename(index=features_glaucoma).sort_values()

colors = ['#1f77b4' if x != 'Presión Intraocular' else 'red' for x in importances.index]
importances.plot(kind='barh', ax=ax[1], color=colors)
ax[1].set_title("Factores de Riesgo en Glaucoma")
ax[1].set_xlabel("Importancia Relativa")

# C. Curva ROC para evaluación clínica
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
ax[2].plot(fpr, tpr, color='#9467bd', lw=2,
          label=f'AUC = {roc_auc_score(y_test, y_proba):.2f}')
ax[2].plot([0, 1], [0, 1], 'k--')
ax[2].set_xlabel('Tasa Falsos Positivos')
ax[2].set_ylabel('Tasa Verdaderos Positivos')
ax[2].set_title('Curva ROC')
ax[2].legend(loc="lower right")

plt.tight_layout()
plt.show()

# 6. Análisis de casos individuales (para toma de decisiones)
print("\n🧪 PREDICCIÓN PARA NUEVOS PACIENTES:")
nuevos_pacientes = pd.DataFrame({
    'Edad': [60, 45],
    'PIO': [21, 26],
    'Paquimetria': [530, 480],
    'RNFL': [75, 82],
    'Campo_Visual': [-4.2, -1.8]
})

predicciones = best_model.predict_proba(nuevos_pacientes)[:, 1]
for i, prob in enumerate(predicciones):
    print(f"Paciente {i+1}: Riesgo de glaucoma = {prob:.1%} → {'¡Alerta!' if prob > 0.5 else 'Control rutinario'}")